In [ ]:
import scipy.io as scio
import pandas as pd
import os
import numpy as np
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import interpolate
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.utils.data as Data
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from torch import einsum
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneGroupOut
import copy
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
#from sklearn import preprocessing
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from tqdm import tqdm, trange
from torcheeg.datasets.constants.emotion_recognition import format_region_channel_list
import random
import mne

from torcheeg.datasets.constants.emotion_recognition.deap import DEAP_GENERAL_REGION_LIST
from torcheeg.models import (EEGNet, FBCNet, TSCeption)
from torcheeg.models import DGCNN, ArjunViT, STNet, FBCNet, GRU, LSTM, LGGNet, MTCNN


In [ ]:
import numpy as np
EEG_data=np.load('multimodal_data/data_epochs/EEG_data.npy')
ECG_data=np.load('multimodal_data/data_epochs/ECG_data.npy')
EDA_data=np.load('multimodal_data/data_epochs/EDA_data.npy')
LABEL_DEC=np.load('multimodal_data/data_epochs/label_dec.npy')
LABEL_EMO=np.load('multimodal_data/data_epochs/label_emo.npy')


In [ ]:
def set_seed_everywhere(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

In [ ]:
import torch
#Shuffle the data
set_seed_everywhere(3407)
index=np.random.permutation(EEG_data.shape[0])
print(index)
EEG_data=EEG_data[index]
delete_index=[32,42,64,65,66]
EEG_data=np.delete(EEG_data,delete_index,axis=1)

ECG_data=ECG_data[index]
EDA_data=EDA_data[index]
dec_label=LABEL_DEC[index]
emo_label=LABEL_EMO[index]

In [ ]:

eegdata_train=EEG_data[:int(np.floor(len(EEG_data)*0.8)),:,:875]
ecgdata_train=ECG_data[:int(np.floor(len(ECG_data)*0.8)),:7000]
edadata_train=EDA_data[:int(np.floor(len(EDA_data)*0.8)),:7000]
declabel_train=dec_label[:int(np.floor(len(EEG_data)*0.8))]
emolabel_train=emo_label[:int(np.floor(len(EEG_data)*0.8))]

eegdata_val=EEG_data[int(np.floor(len(EEG_data)*0.8)):np.floor(len(EEG_data)*0.9)),:,:875]
ecgdata_val=ECG_data[int(np.floor(len(ECG_data)*0.8)):np.floor(len(ECG_data)*0.9)),:7000]
edadata_val=EDA_data[int(np.floor(len(EDA_data)*0.8)):np.floor(len(EDA_data)*0.9)),:7000]
emolabel_val=emo_label[int(np.floor(len(EEG_data)*0.8)):np.floor(len(EEG_data)*0.9))]
declabel_val=dec_label[int(np.floor(len(EEG_data)*0.8)):np.floor(len(EEG_data)*0.9))]

eegdata_test=EEG_data[int(np.floor(len(EEG_data)*0.9)):,:,:875]
ecgdata_test=ECG_data[int(np.floor(len(ECG_data)*0.9)):,:7000]
edadata_test=EDA_data[int(np.floor(len(EDA_data)*0.9)):,:7000]
emolabel_test=emo_label[int(np.floor(len(EEG_data)*0.9)):]
declabel_test=dec_label[int(np.floor(len(EEG_data)*0.9)):]


In [ ]:
import numpy as np


np.save("dataset_partition/Train_eeg.npy", eegdata_train)
np.save("dataset_partition/Train_ecg.npy", ecgdata_train)
np.save("dataset_partition/Train_eda.npy", edadata_train)
np.save("dataset_partition/Train_declabel.npy", declabel_train)
np.save("dataset_partition/Train_emolabel.npy", emolabel_train)


np.save("dataset_partition/Val_eeg.npy", eegdata_val)
np.save("dataset_partition/Val_ecg.npy", ecgdata_val)
np.save("dataset_partition/Val_eda.npy", edadata_val)
np.save("dataset_partition/Val_declabel.npy", declabel_val)
np.save("dataset_partition/Val_emolabel.npy", emolabel_val)


np.save("dataset_partition/Test_eeg.npy", eegdata_test)
np.save("dataset_partition/Test_ecg.npy", ecgdata_test)
np.save("dataset_partition/Test_eda.npy", edadata_test)
np.save("dataset_partition/Test_declabel.npy", declabel_test)
np.save("dataset_partition/Test_emolabel.npy", emolabel_test)


In [ ]:
import numpy as np
import torch

import numpy as np


train_eeg_data = np.load("dataset_partition/Train_eeg.npy")
train_ecg_data = np.load("dataset_partition/Train_ecg.npy")
train_eda_data = np.load("dataset_partition/Train_eda.npy")
declabel_train = np.load("dataset_partition/Train_declabel.npy")
emolabel_train = np.load("dataset_partition/Train_emolabel.npy")

val_eeg_data = np.load("dataset_partition/Val_eeg.npy")
val_ecg_data = np.load("dataset_partition/Val_ecg.npy")
val_eda_data = np.load("dataset_partition/Val_eda.npy")
val_declabel = np.load("dataset_partition/Val_declabel.npy")
val_emolabel = np.load("dataset_partition/Val_emolabel.npy")

test_eeg_data = np.load("dataset_partition/Test_eeg.npy")
test_ecg_data = np.load("dataset_partition/Test_ecg.npy")
test_eda_data = np.load("dataset_partition/Test_eda.npy")
test_declabel = np.load("dataset_partition/Test_declabel.npy")
test_emolabel = np.load("dataset_partition/Test_emolabel.npy")


In [4]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch
import torch.utils.data as Data
train_eeg_data = torch.from_numpy(train_eeg_data).float().reshape(-1,1,62,875)
train_ecg_data = torch.from_numpy(train_ecg_data).float()
train_eda_data = torch.from_numpy(train_eda_data).float()
train_dec_label = torch.from_numpy(declabel_train).long()
train_emo_label = torch.from_numpy(emolabel_train).long()

val_eeg_data = torch.from_numpy(val_eeg_data).float().reshape(-1, 1, 62, 875)
val_ecg_data = torch.from_numpy(val_ecg_data).float()
val_eda_data = torch.from_numpy(val_eda_data).float()
val_dec_label = torch.from_numpy(val_declabel).long()
val_emo_label = torch.from_numpy(val_emolabel).long()

test_eeg_data = torch.from_numpy(test_eeg_data).float().reshape(-1,1,62,875)
test_ecg_data = torch.from_numpy(test_ecg_data).float()
test_eda_data = torch.from_numpy(test_eda_data).float()
test_dec_label = torch.from_numpy(test_declabel).long()
test_emo_label= torch.from_numpy(test_emolabel).long()


In [5]:
train_eeg_data.shape,train_ecg_data.shape, train_eda_data.shape, train_dec_label.shape, train_emo_label.shape

(torch.Size([1698, 1, 62, 875]),
 torch.Size([1698, 7000]),
 torch.Size([1698, 7000]),
 torch.Size([1698]),
 torch.Size([1698]))

In [ ]:
#downsampling of ecg and eda
window_size = 8

avg_pool = torch.nn.AvgPool1d(kernel_size=window_size, stride=window_size)

train_ecg_data = avg_pool(train_ecg_data)
train_eda_data = avg_pool(train_eda_data)

val_ecg_data = avg_pool(val_ecg_data)
val_eda_data = avg_pool(val_eda_data)

test_ecg_data = avg_pool(test_ecg_data)
test_eda_data = avg_pool(test_eda_data)

In [7]:
train_eeg_data.shape,train_ecg_data.shape, train_eda_data.shape, train_dec_label.shape, train_emo_label.shape

(torch.Size([1698, 1, 62, 875]),
 torch.Size([1698, 875]),
 torch.Size([1698, 875]),
 torch.Size([1698]),
 torch.Size([1698]))

In [8]:
val_eeg_data.shape,val_ecg_data.shape, val_eda_data.shape, val_dec_label.shape, val_emo_label.shape

(torch.Size([212, 1, 62, 875]),
 torch.Size([212, 875]),
 torch.Size([212, 875]),
 torch.Size([212]),
 torch.Size([212]))

In [9]:
test_eeg_data.shape,test_ecg_data.shape, test_eda_data.shape, test_dec_label.shape, test_emo_label.shape

(torch.Size([213, 1, 62, 875]),
 torch.Size([213, 875]),
 torch.Size([213, 875]),
 torch.Size([213]),
 torch.Size([213]))

In [ ]:
##asf and mtf features of ecg and eda in train set
from pyts.image import GramianAngularField, MarkovTransitionField

gasf = GramianAngularField(image_size=64, method='summation')
ecg_gasf_train = gasf.fit_transform(train_ecg_data)
eda_gasf_train = gasf.fit_transform(train_eda_data)
mtf = MarkovTransitionField(image_size=64)
ecg_mtf_train = mtf.fit_transform(train_ecg_data)
eda_mtf_train = mtf.fit_transform(train_eda_data)

In [ ]:
#gasf and mtf features of ecg and eda in val set
from pyts.image import GramianAngularField, MarkovTransitionField

gasf = GramianAngularField(image_size=64, method='summation')
ecg_gasf_val = gasf.fit_transform(val_ecg_data)
eda_gasf_val = gasf.fit_transform(val_eda_data)
mtf = MarkovTransitionField(image_size=64)
ecg_mtf_val = mtf.fit_transform(val_ecg_data)
eda_mtf_val = mtf.fit_transform(val_eda_data)

In [ ]:
#gasf and mtf features of ecg and eda in test set
from pyts.image import GramianAngularField, MarkovTransitionField

gasf = GramianAngularField(image_size=64, method='summation')
ecg_gasf_test = gasf.fit_transform(test_ecg_data)
eda_gasf_test = gasf.fit_transform(test_eda_data)
mtf = MarkovTransitionField(image_size=64)
ecg_mtf_test = mtf.fit_transform(test_ecg_data)
eda_mtf_test = mtf.fit_transform(test_eda_data)

In [ ]:
#RP feature of ecg and eda in train set
import numpy as np
import cv2
from sklearn.metrics.pairwise import pairwise_distances

def recurrence_plot(s, eps=None, steps=None):
    if eps is None: eps = 0.1
    if steps is None: steps = 10
    d = pairwise_distances(s[:, None])
    d = d / eps
    d[d > steps] = steps
    return d / 5.0 - 1

def compress_image(image, target_size):
    compressed_image = cv2.resize(image, (target_size, target_size), interpolation=cv2.INTER_LINEAR)
    return compressed_image


ecg_rp_train = []
eda_rp_train = []

## target image size
target_size = 64

for i in range(len(train_ecg_data)):
    result = recurrence_plot(train_ecg_data[i], steps=10)
    compressed_result = compress_image(result, target_size)
    ecg_rp_train.append(compressed_result)

for i in range(len(train_eda_data)):
    result = recurrence_plot(train_eda_data[i], steps=10)
    compressed_result = compress_image(result, target_size)
    eda_rp_train.append(compressed_result)

ecg_rp_train = np.array(ecg_rp_train)
eda_rp_train = np.array(eda_rp_train)

print(ecg_rp_train.shape)
print(eda_rp_train.shape)


In [ ]:
#RP feature of ecg and eda in val set
import numpy as np
import cv2
from sklearn.metrics.pairwise import pairwise_distances

def recurrence_plot(s, eps=None, steps=None):
    if eps is None: eps = 0.1
    if steps is None: steps = 10
    d = pairwise_distances(s[:, None])
    d = d / eps
    d[d > steps] = steps
    return d / 5.0 - 1

def compress_image(image, target_size):

    compressed_image = cv2.resize(image, (target_size, target_size), interpolation=cv2.INTER_LINEAR)
    return compressed_image

ecg_rp_val = []
eda_rp_val = []

# target image size
target_size = 64

for i in range(len(val_ecg_data)):
    result = recurrence_plot(val_ecg_data[i], steps=10)
    compressed_result = compress_image(result, target_size)
    ecg_rp_val.append(compressed_result)

for i in range(len(val_eda_data)):
    result = recurrence_plot(val_eda_data[i], steps=10)
    compressed_result = compress_image(result, target_size)
    eda_rp_val.append(compressed_result)

ecg_rp_val = np.array(ecg_rp_val)
eda_rp_val = np.array(eda_rp_val)

print(ecg_rp_val.shape)
print(eda_rp_val.shape)


In [ ]:
#RP feature of ecg and eda in test set
import numpy as np
import cv2
from sklearn.metrics.pairwise import pairwise_distances

def recurrence_plot(s, eps=None, steps=None):
    if eps is None: eps = 0.1
    if steps is None: steps = 10
    d = pairwise_distances(s[:, None])
    d = d / eps
    d[d > steps] = steps
    return d / 5.0 - 1

def compress_image(image, target_size):
    compressed_image = cv2.resize(image, (target_size, target_size), interpolation=cv2.INTER_LINEAR)
    return compressed_image

ecg_rp_test = []
eda_rp_test = []

# target image size
target_size = 64

for i in range(len(test_ecg_data)):
    result = recurrence_plot(test_ecg_data[i], steps=10)
    compressed_result = compress_image(result, target_size)
    ecg_rp_test.append(compressed_result)

for i in range(len(test_eda_data)):
    result = recurrence_plot(test_eda_data[i], steps=10)
    compressed_result = compress_image(result, target_size)
    eda_rp_test.append(compressed_result)

ecg_rp_test = np.array(ecg_rp_test)
eda_rp_test = np.array(eda_rp_test)

print(ecg_rp_test.shape)
print(eda_rp_test.shape)


In [ ]:
import matplotlib.pyplot as plt

# plot 2D-image features of ecg
fig, axs = plt.subplots(1, 3, figsize=(12, 4))


axs[0].imshow(ecg_gasf_train[1], cmap='gray', origin='lower')
axs[0].set_title('GASF Image')


axs[1].imshow(ecg_mtf_train[1], cmap='gray', origin='lower')
axs[1].set_title('MTF Image')


axs[2].imshow(ecg_rp_train[1], cmap='gray', origin='lower')
axs[2].set_title('RP Image')


plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# plot 2D-image features of eda
fig, axs = plt.subplots(1, 3, figsize=(12, 4))


axs[0].imshow(eda_gasf_train[1], cmap='gray', origin='lower')
axs[0].set_title('GASF Image')



axs[1].imshow(eda_mtf_train[1], cmap='gray', origin='lower')
axs[1].set_title('MTF Image')


axs[2].imshow(eda_rp_train[1], cmap='gray', origin='lower')
axs[2].set_title('RP Image')


plt.tight_layout()
plt.show()


In [27]:
import torch
train_eda_image = torch.from_numpy(train_eda_image)
val_eda_image = torch.from_numpy(val_eda_image)
test_eda_image = torch.from_numpy(test_eda_image)

train_ecg_image = torch.from_numpy(train_ecg_image)
val_ecg_image = torch.from_numpy(val_ecg_image)
test_ecg_image = torch.from_numpy(test_ecg_image)


In [ ]:
import mne
eeg_data_now = mne.io.read_raw_eeglab('/data/zhaoyanyan/notebook_code/2024-4-22备份/model/Multimodal-Transformer-master/src/01102201.set', preload=True)
channels_to_remove = ['M1', 'M2','HEO','VEO','Trigger']

eeg_data_now.pick_channels(ch_names=[ch for ch in eeg_data_now.ch_names if ch not in channels_to_remove])

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,67 points
Good channels,62 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.00 Hz
Lowpass,500.00 Hz


In [ ]:
from torcheeg.datasets.constants.emotion_recognition import format_region_channel_list
'''
To Grid
'''
LOCATION_LIST = [
    [ '-', '-', '-', 'FP1', 'FPZ', 'FP2', '-', '-', '-'],
    [ '-', '-', '-', 'AF3', '-', 'AF4', '-', '-', '-'],
    [ 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'],
    [ 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'],
    [ 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'],
    [ 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'],
    [ 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'],
    [ '-', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '-'],
    [ '-', '-', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '-', '-' ]
]
HEMISPHERE_LIST = [
    ['F7', 'F5', 'F3', 'F1'], 
    ['F2', 'F4', 'F6', 'F8'],
    [ 'FC5', 'FC3', 'FC1'],
    ['FC2', 'FC4', 'FC6'], 
    ['FP1',  'AF3'], 
    ['FP2', 'AF4', ],
    ['FPZ', 'FZ', 'FCZ', 'CZ', 'CPZ', 'PZ', 'POZ', 'OZ'],
    ['C5', 'C3', 'C1'], ['C2', 'C4', 'C6'], ['CP5', 'CP3', 'CP1'],
    ['CP2', 'CP4', 'CP6'], 
    ['P7', 'P5', 'P3', 'P1'], 
    ['P2', 'P4', 'P6', 'P8'],
    ['PO7','PO5', 'PO3', 'O1', 'CB1'], 
    ['PO4', 'PO6', 'PO8', 'O2', 'CB2'], 
    ['FT7','T7', 'TP7'],
    ['FT8','T8', 'TP8' ]
]
CHANNELS_LIST = eeg_data_now.ch_names

'''
local-global structure
'''
GENERAL_REGION_LIST = format_region_channel_list(CHANNELS_LIST, LOCATION_LIST)     #按行分

HEMISPHERE_REGION_LIST = format_region_channel_list(CHANNELS_LIST, HEMISPHERE_LIST)            #按区域分


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_size):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(in_size*2, 64),
            nn.LeakyReLU(inplace=True),
            nn.Linear(64, in_size*2),
        )
        self.Linear1= nn.Linear(in_size*2,in_size)
    def forward(self, x):
        x= x + self.block(x)
        x= self.Linear1(x)
        return x


class graph11_new(nn.Module):

    def __init__(self, in_size, hidden = 50, dropout=0.5):

        super(graph11_new, self).__init__()
        self.norm = nn.BatchNorm1d(in_size)
        self.norm2 = nn.BatchNorm1d(in_size*3)
        self.drop = nn.Dropout(p=dropout)
      #  self.graph = nn.Linear(in_size*2, in_size)

        self.graph_fusion = nn.Sequential(
            ResidualBlock(in_size),
            nn.Tanh()
        )


        self.graph_fusion2 = nn.Sequential(
            ResidualBlock(in_size),
            nn.Tanh()
        )

        self.attention = nn.Linear(in_size, 1)
        self.linear_1 = nn.Linear(in_size*3, hidden)
        self.linear_2 = nn.Linear(hidden, hidden)
        self.hidden = hidden
        self.in_size = in_size

      #  self.u1 = Parameter(torch.Tensor(in_size, in_size).cuda())
     #   xavier_normal(self.u1)

    def forward(self, a1,v1,l1):
       
        ###################### unimodal layer  ##########################
        sa = F.sigmoid(self.attention(a1))
        sv = F.sigmoid(self.attention(v1))
        sl = F.sigmoid(self.attention(l1))

        total_weights = torch.cat([sa, sv, sl],1)

        unimodal_a = (sa.expand(a1.size(0),self.in_size))
        unimodal_v = (sv.expand(a1.size(0),self.in_size))
        unimodal_l = (sl.expand(a1.size(0),self.in_size))
        sa = sa.squeeze()
        sl = sl.squeeze()
        sv = sv.squeeze()
        unimodal = (unimodal_a * a1 + unimodal_v * v1 + unimodal_l * l1)/3

        ##################### bimodal layer ############################
        a = F.softmax(a1, 1)
        v = F.softmax(v1, 1)
        l = F.softmax(l1, 1)
        sav = (1/(torch.matmul(a.unsqueeze(1), v.unsqueeze(2)).squeeze() +0.5) *(sa+sv))
        sal = (1/(torch.matmul(a.unsqueeze(1), l.unsqueeze(2)).squeeze() +0.5) *(sa+sl))
        svl = (1/(torch.matmul(v.unsqueeze(1), l.unsqueeze(2)).squeeze() +0.5) *(sl+sv))
     #   print('sav',sav.shape)
        normalize = torch.cat([sav.unsqueeze(1), sal.unsqueeze(1), svl.unsqueeze(1)],1)
        normalize = F.softmax(normalize,1)
        total_weights = torch.cat([total_weights,normalize],1)
    #    print('normalize',normalize.shape)
     #   print((normalize[:,0].unsqueeze(1).expand(a.size(0),self.in_size)).shape,'shape')
        a_v = F.elu((normalize[:,0].unsqueeze(1).expand(a.size(0), self.in_size)) * self.graph_fusion(torch.cat([a1,v1],1)))
        a_l = F.elu((normalize[:,1].unsqueeze(1).expand(a.size(0), self.in_size)) * self.graph_fusion(torch.cat([a1,l1],1)))
        v_l = F.elu((normalize[:,2].unsqueeze(1).expand(a.size(0), self.in_size)) * self.graph_fusion(torch.cat([v1,l1],1)))
        bimodal = (a_v + a_l + v_l)
    
        ###################### trimodal layer ####################################
        a_v2 = F.softmax(self.graph_fusion(torch.cat([a1,v1],1)), 1)
        a_l2 = F.softmax(self.graph_fusion(torch.cat([a1,l1],1)), 1)
        v_l2 = F.softmax(self.graph_fusion(torch.cat([v1,l1],1)), 1)
        savvl = (1/(torch.matmul(a_v2.unsqueeze(1), v_l2.unsqueeze(2)).squeeze() +0.5) *(sav+svl))
        saavl = (1/(torch.matmul(a_v2.unsqueeze(1), a_l2.unsqueeze(2)).squeeze() +0.5) *(sav+sal))
        savll = (1/(torch.matmul(a_l2.unsqueeze(1), v_l2.unsqueeze(2)).squeeze() +0.5) *(sal+svl))
        savl = (1/(torch.matmul(a_v2.unsqueeze(1), l.unsqueeze(2)).squeeze() +0.5) *(sav+sl))
        salv = (1/(torch.matmul(a_l2.unsqueeze(1), v.unsqueeze(2)).squeeze() +0.5) *(sal+sv))
        svla = (1/(torch.matmul(v_l2.unsqueeze(1), a.unsqueeze(2)).squeeze() +0.5) *(sa+svl))

        normalize2 = torch.cat([savvl.unsqueeze(1), saavl.unsqueeze(1), savll.unsqueeze(1), savl.unsqueeze(1), salv.unsqueeze(1), svla.unsqueeze(1)],1)
        normalize2 = F.softmax(normalize2,1)
        total_weights = torch.cat([total_weights,normalize2],1)
       # print((normalize2[:,0].unsqueeze(1).expand(a.size(0),self.in_size)).shape,'shape')
        avvl = F.elu((normalize2[:,0].unsqueeze(1).expand(a.size(0),self.in_size)) * self.graph_fusion2(torch.cat([a_v,v_l],1)))
        aavl = F.elu((normalize2[:,1].unsqueeze(1).expand(a.size(0),self.in_size)) * self.graph_fusion2(torch.cat([a_v,a_l],1)))
        avll = F.elu((normalize2[:,2].unsqueeze(1).expand(a.size(0),self.in_size)) * self.graph_fusion2(torch.cat([v_l,a_l],1)))
        avl = F.elu((normalize2[:,3].unsqueeze(1).expand(a.size(0),self.in_size)) * self.graph_fusion2(torch.cat([a_v,l1],1)))
        alv = F.elu((normalize2[:,4].unsqueeze(1).expand(a.size(0),self.in_size)) * self.graph_fusion2(torch.cat([a_l,v1],1)))
        vla = F.elu((normalize2[:,5].unsqueeze(1).expand(a.size(0),self.in_size)) * self.graph_fusion2(torch.cat([v_l,a1],1)))
        trimodal = (avvl + aavl + avll + avl + alv + vla)
        fusion = torch.cat([unimodal,bimodal],1)
        fusion = torch.cat([fusion,trimodal],1)        
        fusion = self.norm2(fusion)
     #   fusion = self.drop(fusion)
        y_1 = F.tanh(self.linear_1(fusion))
        y_1 = F.tanh(self.linear_2(y_1))

        return y_1

In [ ]:
from PIL import Image

import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter
from torchvision.models import resnet18
import torchvision.transforms as transforms


class ChannelAttention(nn.Module):
    def __init__(self, in_channelsnel, ratio=2):
        super(ChannelAttention, self).__init__()
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(in_channelsnel,
                             in_channelsnel // ratio,
                             1,
                             bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_channelsnel // ratio,
                             in_channelsnel,
                             1,
                             bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_pool_out = self.avg_pool(x)
        max_pool_out = self.max_pool(x)
        avg_pool_out = self.fc2(self.relu1(self.fc1(avg_pool_out)))
        max_pool_out = self.fc2(self.relu1(self.fc1(max_pool_out)))
        out = max_pool_out + avg_pool_out
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'Kernel size must be 3 or 7.'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        max_pool_out, _ = torch.max(x, dim=1, keepdim=True)
        avg_pool_out = torch.mean(x, dim=1, keepdim=True)
        out = torch.cat([avg_pool_out, max_pool_out], dim=1)
        out = self.conv1(out)
        return self.sigmoid(out)


class CBAMBlock(nn.Module):
    def __init__(self, in_channelsnel, ratio=2, kernel_size=7):
        super(CBAMBlock, self).__init__()
        self.cha_att = ChannelAttention(in_channelsnel, ratio=ratio)
        self.spa_att = SpatialAttention(kernel_size=kernel_size)

    def forward(self, x):
        out = x * self.cha_att(x)
        out = out * self.spa_att(out)
        return out


class PowerLayer(nn.Module):
    def __init__(self, kernel_size, stride):
        super(PowerLayer, self).__init__()
        self.pooling = nn.AvgPool2d(kernel_size=(1, kernel_size),
                                    stride=(1, stride))

    def forward(self, x):
        return torch.log(self.pooling(x.pow(2)))


class Aggregator():
    def __init__(self, region_list):
        self.region_list = region_list

    def forward(self, x):
        output = []
        for region_index in range(len(self.region_list)):
            region_x = x[:, self.region_list[region_index], :]
            aggr_region_x = torch.mean(region_x, dim=1)
            output.append(aggr_region_x)
        return torch.stack(output, dim=1)


class GraphConvolution(Module):
    def __init__(self, in_channels, out_channels, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.weight = Parameter(torch.FloatTensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(
                torch.zeros((1, 1, out_channels), dtype=torch.float32))
        else:
            self.register_parameter('bias', None)
        nn.init.xavier_uniform_(self.weight, gain=1.414)

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x, adj):
        output = torch.matmul(x, self.weight) - self.bias
        output = F.relu(torch.matmul(adj, output))
        return output


class GRF_MMPS(nn.Module):
    def __init__(self,
                 region_list,
                 in_channels: int = 1,
                 num_electrodes: int = 32,
                 chunk_size: int = 128,
                 sampling_rate: int = 128,
                 num_T: int = 64,
                 hid_channels: int = 32,
                 dropout: float = 0.5,
                 pool_kernel_size: int = 16,
                 pool_stride: int = 4,
                 num_classes_dec: int = 2,
                num_classes_emo:int = 4):
        super(GRF-MMPS, self).__init__()
        self.region_list = region_list
        self.inception_window = [0.5, 0.25, 0.125]

        self.num_classes_dec = num_classes_dec
        self.num_classes_emo= num_classes_emo
        self.in_channels = in_channels
        self.num_electrodes = num_electrodes
        self.chunk_size = chunk_size
        self.sampling_rate = sampling_rate
        self.num_T = num_T
        self.hid_channels = hid_channels
        self.dropout = dropout
        self.pool_kernel_size = pool_kernel_size
        self.pool_stride = pool_stride
        self.in_channels = in_channels
        self.num_electrodes = num_electrodes
        self.graph11_new=graph11_new(512,50,0.5)

        self.t_block1 = self.temporal_block(
            self.in_channels, self.num_T,
            (1, int(self.inception_window[0] * self.sampling_rate)),
            self.pool_kernel_size, self.pool_stride)
        self.t_block2 = self.temporal_block(
            self.in_channels, self.num_T,
            (1, int(self.inception_window[1] * self.sampling_rate)),
            self.pool_kernel_size, self.pool_stride)
        self.t_block3 = self.temporal_block(
            self.in_channels, self.num_T,
            (1, int(self.inception_window[2] * self.sampling_rate)),
            self.pool_kernel_size, self.pool_stride)

        self.bn_t1 = nn.BatchNorm2d(self.num_T)
        self.bn_t2 = nn.BatchNorm2d(self.num_T)

        self.cbam = CBAMBlock(num_electrodes)

        self.conv1x1 = nn.Sequential(
            nn.Conv2d(num_T, num_T, kernel_size=(1, 1), stride=(1, 1)),
            nn.LeakyReLU(), nn.AvgPool2d((1, 2)))

        self.avg_pool = nn.AvgPool2d((1, 2))

        feature_dim = self.feature_dim
        self.local_filter_weight = nn.Parameter(torch.FloatTensor(
            self.num_electrodes, feature_dim),
                                                requires_grad=True)
        self.local_filter_bias = nn.Parameter(torch.zeros(
            (1, self.num_electrodes, 1), dtype=torch.float32),
                                              requires_grad=True)

        self.aggregate = Aggregator(self.region_list)
        num_region = len(self.region_list)

        self.global_adj = nn.Parameter(torch.FloatTensor(
            num_region, num_region),
                                       requires_grad=True)

        self.bn_g1 = nn.BatchNorm1d(num_region)
        self.bn_g2 = nn.BatchNorm1d(num_region)

        self.gcn = GraphConvolution(feature_dim, hid_channels)
        
        

        self.fc_dec = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(50, num_classes_dec))
        
        self.fc_emo = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(50, num_classes_emo))

        nn.init.xavier_uniform_(self.local_filter_weight)
        nn.init.xavier_uniform_(self.global_adj)
        
        self.fc_512 = nn.Linear(int(num_region * hid_channels), 512)
        
        self.bn_512 = nn.BatchNorm1d(512)
        
        #ecg
        self.resnet18_ecg=resnet18(pretrained=True)
        in_features_ecg = self.resnet18_ecg.fc.in_features
        
        self.resnet18_ecg.fc = nn.Identity()
        self.ecg_fc = nn.Linear(in_features_ecg, 32)
        self.bn_ecg = nn.BatchNorm1d(512)
        self.transform_ecg = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        #eda
        self.resnet18_eda=resnet18(pretrained=True)
        in_features_eda = self.resnet18_eda.fc.in_features
        self.resnet18_eda.fc = nn.Identity()
        self.eda_fc = nn.Linear(in_features_eda, 32)
        self.bn_eda = nn.BatchNorm1d(512)
        self.transform_eda = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def temporal_block(self, in_channels, out_channels, kernel_size,
                       pool_kernel_size, pool_stride):
        return nn.Sequential(
            nn.Conv2d(in_channels,
                      out_channels,
                      kernel_size=kernel_size,
                      stride=(1, 1)),
            PowerLayer(kernel_size=pool_kernel_size, stride=pool_stride))

    def forward(self, x_eeg,x_ecg,x_eda):
        t1 = self.t_block1(x_eeg)
        t2 = self.t_block2(x_eeg)
        t3 = self.t_block3(x_eeg)
        x_eeg = torch.cat((t1, t2, t3), dim=-1)

        x_eeg = self.bn_t1(x_eeg)

        x_eeg = x_eeg.permute(0, 2, 1, 3)
        x_eeg = self.cbam(x_eeg)
        x_eeg = self.avg_pool(x_eeg)

        x_eeg = x_eeg.flatten(start_dim=2)
#         print(x_eeg.shape)
        x_eeg = self.local_filter(x_eeg)
        x_eeg = self.aggregate.forward(x_eeg)
        adj = self.get_adj(x_eeg)
        x_eeg = self.bn_g1(x_eeg)

        x_eeg = self.gcn(x_eeg, adj)
        x_eeg = self.bn_g2(x_eeg)
        x_eeg = x_eeg.view(x_eeg.shape[0], -1)
        x_eeg = self.fc_512(x_eeg)
#         x_eeg = self.bn_512(x_eeg)
        
        #ecg、eda feature extraction
#         x_ecg = x_ecg.permute(0, 3, 1, 2)
        x_ecg = x_ecg.cpu().numpy()
        ecg_input_pil = [Image.fromarray((img * 255).astype(np.uint8)) for img in x_ecg]  # 转换为 PIL 图像
        ecg_input_transformed = [self.transform_ecg(img) for img in ecg_input_pil]
        x_ecg = torch.stack(ecg_input_transformed).to('cuda:0')  # 转换为 batch tensor
#         x_ecg = self.transform_ecg(x_ecg)
        x_ecg = self.resnet18_ecg(x_ecg)
#         x_ecg = self.ecg_fc(x_ecg)
#         x_ecg = self.bn_ecg(x_ecg)
        
#         x_eda = x_eda.permute(0, 3, 1, 2)
#         x_eda = self.transform_eda(x_eda)
        x_eda = x_eda.cpu().numpy()
        eda_input_pil = [Image.fromarray((img * 255).astype(np.uint8)) for img in x_eda]  # 转换为 PIL 图像
        eda_input_transformed = [self.transform_eda(img) for img in eda_input_pil]
        x_eda = torch.stack(eda_input_transformed).to('cuda:0')  # 转换为 batch tensor
        x_eda = self.resnet18_eda(x_eda)
#         x_eda = self.eda_fc(x_eda)
#         x_eda = self.bn_eda(x_eda)
        
        x_multi = self.graph11_new(x_eeg,x_ecg,x_eda)
        
        x_multi1 = self.fc_dec(x_multi)
        x_multi2= self.fc_emo(x_multi)
        return  x_multi1, x_multi2,x_multi

    @property
    def feature_dim(self):
        mock_eeg = torch.randn(
            (1, self.in_channels, self.num_electrodes, self.chunk_size))

        t1 = self.t_block1(mock_eeg)
        t2 = self.t_block2(mock_eeg)
        t3 = self.t_block3(mock_eeg)
        mock_eeg = torch.cat((t1, t2, t3), dim=-1)

        mock_eeg = self.bn_t1(mock_eeg)
        mock_eeg = self.conv1x1(mock_eeg)
        mock_eeg = self.bn_t2(mock_eeg)
        mock_eeg = mock_eeg.permute(0, 2, 1, 3)
        mock_eeg = mock_eeg.flatten(start_dim=2)
        return mock_eeg.shape[-1]

    def local_filter(self, x):
        w = self.local_filter_weight.unsqueeze(0).repeat(x.shape[0], 1, 1)
        x = F.relu(torch.mul(x, w) - self.local_filter_bias)
        return x

    def get_adj(self, x, self_loop=True):
        adj = torch.bmm(x, x.permute(0, 2, 1))
        num_nodes = adj.shape[-1]
        adj = F.relu(adj * (self.global_adj + self.global_adj.transpose(1, 0)))
        if self_loop:
            adj = adj + torch.eye(num_nodes).to(x.device)
        rowsum = torch.sum(adj, dim=-1)
        mask = torch.zeros_like(rowsum)
        mask[rowsum == 0] = 1
        rowsum += mask
        d_inv_sqrt = torch.pow(rowsum, -0.5)
        d_mat_inv_sqrt = torch.diag_embed(d_inv_sqrt)
        adj = torch.bmm(torch.bmm(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)
        return adj

In [41]:
import numpy as np
import torch
from torch.nn.modules.module import Module
import torch.nn.functional as F

class SmoothCrossEntropyLoss(Module):
    def __init__(self, label_smoothing=0.0):
        super(SmoothCrossEntropyLoss, self).__init__()
        self.label_smoothing = label_smoothing

    def forward(self, input, target):
        if len(target.size()) == 1:
            target = F.one_hot(target, num_classes=input.size(-1)).float().to(input.device)
        
        if self.label_smoothing > 0.0:
            num_classes = input.size(-1)
            smooth_value = self.label_smoothing / num_classes
            target = target * (1.0 - self.label_smoothing) + smooth_value
        
        log_probs = F.log_softmax(input, dim=-1)
        loss = -torch.sum(target * log_probs, dim=-1)

        return loss.mean()

In [ ]:
import os
import csv

def log_training_progress(epoch, batch, loss, accuracy, hyperparameters,log_file):
    with open(log_file, 'a', newline='') as csvfile:
        fieldnames = ['Epoch', 'Batch', 'Loss', 'Accuracy', 'Hyperparameters']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        
        if csvfile.tell() == 0:
            writer.writeheader()

        
        writer.writerow({'Epoch': epoch, 'Batch': batch, 'Loss': loss, 'Accuracy': accuracy,
                         'Hyperparameters': hyperparameters})

def log_final_results(accuracy, loss, val_accuracy, val_precision, val_recall, val_f1, hyperparameters,log_file):
    with open(log_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)

        
        writer.writerow(['Final Results'])
        writer.writerow(['Accuracy', accuracy])
        writer.writerow(['Loss', loss])
        writer.writerow(['Validation Accuracy', val_accuracy])
        writer.writerow(['Validation Precision', val_precision])
        writer.writerow(['Validation recall', val_recall])
        writer.writerow(['Validation F1', val_f1])
        
#         writer.writerow(['Validation Loss', val_loss])

        
        writer.writerow(['Hyperparameters'])
        for key, value in hyperparameters.items():
            writer.writerow([key, value])
        writer.writerow(['-'*30])
#         
#         writer.writerow(['Model Summary'])
#         writer.writerow([model_summary])

def create_experiment_folder(experiment_folder):
    os.makedirs(experiment_folder, exist_ok=True)
    return experiment_folder



In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_set = TensorDataset(train_eeg_data, train_ecg_image, train_eda_image, train_dec_label, train_emo_label)
train_loader = Data.DataLoader(train_set, batch_size=32)


val_set = TensorDataset(val_eeg_data, val_ecg_image, val_eda_image, val_dec_label, val_emo_label)
val_loader = DataLoader(val_set, batch_size=32)


test_set = TensorDataset(test_eeg_data, test_ecg_image, test_eda_image, test_dec_label, test_emo_label)
test_loader = DataLoader(test_set, batch_size=32)

    

In [ ]:
from sklearn.model_selection import KFold
from tqdm import tqdm, trange
from torch.autograd import Variable
import os
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
from datetime import datetime


def multi_learning(alpha,beta,learning_rate1,epoch_size,dec_smoothing,emo_smoothing,iter_times):
    # for i in range(len(model_name_list)):
    #     model_name=model_name_list[i]

    current_time = datetime.now()
    
    formatted_time = current_time.strftime("%Y-%m-%d_%H-%M-%S")
    print(formatted_time)
    
    experiment_folder = create_experiment_folder('experiment_result/'+formatted_time)
    training_log_file = os.path.join(experiment_folder, 'training_log.csv')
    final_results_log_file = os.path.join(experiment_folder, 'final_results_log.csv')

    
    
    criterion1 = SmoothCrossEntropyLoss(label_smoothing=dec_smoothing)
    criterion2 = SmoothCrossEntropyLoss(label_smoothing=emo_smoothing)

#     learning_rate = 0.0005
#     epoch_size = 50
    device = torch.device("cuda:0")
    ##############################
    model_name = 'MMPS-EDNet'

    save = True
    
    
    hyperparameters={'alpha':alpha,'beta':beta,'epoch_size':epoch_size,'learning_rate': learning_rate1, 'batch_size': 32,
                     'dec_smoothing':dec_smoothing,'emo_smoothing':emo_smoothing}
    
    
    
    acc_all = []
    f1_all = []
    recall_all = []
    precision_all = []
    
    
#     kf = KFold(n_splits=k_folder, shuffle=True)
    
    fold = 0
    
    for k in range(iter_times):
        learning_rate=learning_rate1
        fold += 1
        print(f"Fold {fold}")
        
        
        model = MMPS_EDNet(region_list=HEMISPHERE_REGION_LIST,chunk_size=875, num_electrodes=62, hid_channels=32, num_classes_dec=2,num_classes_emo=4).to(device)


        """ Optimizer """
        optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)#, weight_decay=0.0005

        

        

        max_acc=0
        for i in range(epoch_size):                                          #多个epoch
            learning_rate=0.98*learning_rate
            
            loop = tqdm(enumerate(train_loader), total=len(train_loader))
            model.train()


            train_loss = 0.0
            train_acc_task = 0.0
            for step, (x_eeg,x_ecg,x_eda, y1, y2) in loop:
                x_eeg, y1, y2 =  Variable(x_eeg).to(device), Variable(y1).to(device), Variable(y2).to(device)
                x_ecg,x_eda = Variable(x_ecg).to(device), Variable(x_eda).to(device)
                
                optimizer.zero_grad()

                
                pred_task, pred_emo,_ = model(x_eeg,x_ecg,x_eda)
                loss1 = criterion1(pred_task, y1.long())
                loss2 = criterion2(pred_emo,y2.long())
                loss= alpha*loss1 + beta*loss2

                train_loss += loss.item()

                pred_task = torch.max(pred_task, 1)[1]
                train_correct_task = (pred_task == y1).sum()

                train_acc_task += train_correct_task.item()

                loss.backward()
                optimizer.step()
                loop.set_description(f'Epoch [{i+1} / {epoch_size}]')
                loop.set_postfix({
                        'loss' : '{:.6f}'.format(train_loss/len(train_set)),
                        'acc_task' : '{:.6f}'.format(train_acc_task*100/len(train_set))
                                                    })
                
                log_training_progress(i, step, train_loss/len(train_set), train_acc_task*100/len(train_set), hyperparameters, training_log_file)
#                 if i+1 == epoch_size and save == True:   
#                     model_path = f'./model_Multi/%s_a=%f_b=%f_%d.pkl' % (model_name,alpha,beta,k+1)  #文件夹名称
#                     os.makedirs('./model_Multi', exist_ok=True)   #创建文件夹
#                     state = {'model':model.state_dict()
#                             }
#                     torch.save(state,model_path)
                    
            prob_all = []
            label_all = []
            with torch.no_grad():
                model.eval()  
                for x_eeg,x_ecg,x_eda, y1, y2 in val_loader:
                    x_eeg, y1, y2 =  Variable(x_eeg).to(device), Variable(y1).to(device), Variable(y2).to(device)
                    x_ecg,x_eda = Variable(x_ecg).to(device), Variable(x_eda).to(device)

                    pred_task, pred_emo,_ = model(x_eeg,x_ecg,x_eda)
                    prob = pred_task.cpu().numpy()
                    prob_all.extend(np.argmax(prob,axis=1)) 
                    label_all.extend(y1.cpu().numpy())
                acc_now = accuracy_score(y_true=prob_all, y_pred=label_all)
                f1_now = f1_score(y_true=prob_all, y_pred=label_all, average="macro")
                recall_now = recall_score(y_true=prob_all, y_pred=label_all, average="macro")
                precision_now = precision_score(y_true=prob_all, y_pred=label_all, average="macro")
            if np.around(acc_now*100, 2)> max_acc:
                max_acc=np.around(acc_now*100, 2)
                max_precision=np.around(precision_now*100, 2)
                max_recall=np.around(recall_now*100, 2)
                max_f1=np.around(f1_now*100, 2)
                #创建文件夹
                model_folder = create_experiment_folder('./model_parameter/'+formatted_time)
                model_path = f'./model_parameter/'+formatted_time+'/%s_fold=%d.pkl' % ('2d-image_multi',fold)  #文件夹名称
                os.makedirs('./model_parameter', exist_ok=True)    
                state = {'model':model.state_dict()}
                
                
        log_final_results(train_loss/len(train_set), train_acc_task*100/len(train_set), max_acc, max_precision, max_recall,
                              max_f1, hyperparameters, final_results_log_file)


        print('accuracy:', max_acc)
        print('precision:', max_precision)
        print('recall:', max_recall)
        print('f1:', max_f1)
        torch.save(state,model_path)
    

    return formatted_time 


In [ ]:
multi_learning(1,0.85,learning_rate1=0.0002,epoch_size=100,dec_smoothing=0,emo_smoothing=0.17,iter_times=5)

In [ ]:
#TEST RESULT
import joblib
device = torch.device("cuda:0")


acc_all = []
f1_all = []
recall_all = []
precision_all = []

fold = 0
for k in range(5):
    fold=fold+1
    load_path = './model_parameter/2024-10-12_23-04-20/%s_fold=%d.pkl' % ('2d-image_multi',fold) 
    state_dict = torch.load(load_path)
    model = MMPS_EDNet(region_list=HEMISPHERE_REGION_LIST,chunk_size=875, num_electrodes=62, hid_channels=32, num_classes_dec=2,num_classes_emo=4).to(device)

    model.load_state_dict(state_dict['model'])
    
    test_set = TensorDataset(test_eeg_data, test_ecg_image, test_eda_image, test_dec_label, test_emo_label)
    test_loader = Data.DataLoader(test_set, batch_size=32)



    prob_all = []
    label_all = []
    with torch.no_grad():
        model.eval()  
        for x_eeg,x_ecg,x_eda, y1, y2 in test_loader:
            x_eeg, y1, y2 =  Variable(x_eeg).to(device), Variable(y1).to(device), Variable(y2).to(device)
            x_ecg,x_eda = Variable(x_ecg).to(device), Variable(x_eda).to(device)

            pred_task, pred_emo,_ = model(x_eeg,x_ecg,x_eda)
            prob = pred_task.cpu().numpy()
            prob_all.extend(np.argmax(prob,axis=1)) #求每一行的最大值索引
            label_all.extend(y1.cpu().numpy())
        acc_now = accuracy_score(y_true=prob_all, y_pred=label_all)
        f1_now = f1_score(y_true=prob_all, y_pred=label_all, average="macro")
        recall_now = recall_score(y_true=prob_all, y_pred=label_all, average="macro")
        precision_now = precision_score(y_true=prob_all, y_pred=label_all, average="macro")
    acc_all.append(acc_now)
    f1_all.append(f1_now)
    recall_all.append(recall_now)
    precision_all.append(precision_now)
    print('accuracy:', np.around(acc_now*100, 2))
    print('precision:', np.around(precision_now*100, 2))
    print('recall:', np.around(recall_now*100, 2))
    print('f1:', np.around(f1_now*100, 2))


